In [8]:
import cv2
import depthai as dai
import time
from depthai_sdk.fps import FPSHandler
import pytesseract
import webbrowser



In [9]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

image = cv2.imread(r"E:\u\sem1\cv\aat4\question2\nitin0210.png",1) 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [11]:

# Create pipeline
pipeline = dai.Pipeline()

# Define source and output
camRgb = pipeline.create(dai.node.ColorCamera)
xoutVideo = pipeline.create(dai.node.XLinkOut)

xoutVideo.setStreamName("video")

# Properties
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
camRgb.setVideoSize(1280,720)

xoutVideo.input.setBlocking(False)
xoutVideo.input.setQueueSize(1)

# Linking
camRgb.video.link(xoutVideo.input)
# Connect to device and start pipeline
start_time = time.time()
x = 1
counter = 0
count=0
with dai.Device(pipeline) as device:

    video = device.getOutputQueue(name="video", maxSize=1, blocking=False)

    while True:
        videoIn = video.get()
        Frame=videoIn.getCvFrame()
        counter+=1
        font = cv2.FONT_HERSHEY_SIMPLEX
        if (time.time() - start_time) >= 1 :
                fps=counter
                counter = 0
                start_time = time.time()
        cv2.putText(Frame, str(fps)+' '+str(camRgb.getVideoSize()), (7, 70), font, 1, (100, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("video", Frame)
        if cv2.waitKey(1)==ord('p'):
            count+=1
            cv2.imwrite(R'E:\u\sem1\cv\aat3\question6\images'+'\image'+str(count)+'.jpg',Frame)
            img = cv2.imread(R'E:\u\sem1\cv\aat3\question6\images\nitin0210.png')
            img = cv2.imread(R'E:\u\sem1\cv\aat4\question2\nitin0210.png')

            ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))
            dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)
            contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            image2 = img.copy() 
            file = open("information.txt", "w+")
            file.write("")
            file.close()
            for cnt in contours:
                x, y, w, h = cv2.boundingRect(cnt)
                rect = cv2.rectangle(image2, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cropped = image2[y:y + h, x:x + w]
                file = open("information.txt", "a")
                text = pytesseract.image_to_string(cropped)
                file.write(text)
                file.close

            detect = cv2.QRCodeDetector()
            url_data, bbox, straight_qrcode = detect.detectAndDecode(img)


        if cv2.waitKey(1) == ord('q'):
            break

In [ ]:
webbrowser.open(val)